# An overview of how a computer looks at an image, and how to use that for further processing

In [ ]:
# Imports!
from tensorflow.keras.datasets import mnist

import matplotlib.pyplot as plt

import skimage.io as io

from sklearn.linear_model import LogisticRegression

import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x) # to make it look better

## First, and example of how a computer breaks an image down

In [ ]:
# Read and Display 

img = io.imread('http://www.colorwiki.com/images/0/0a/Photodisc.png')
plt.figure(figsize=(10,10))
plt.imshow(img);

## How the computer looks at it

In [ ]:
# First, the shape of the photo

print(f'There are {img.shape[0]} pixels in the vertical channel')
print(f'There are {img.shape[1]} pixels in the horizontal')
print(f'There are {img.shape[2]} channels in the "z-axis"')

In [ ]:
# Split into color channels
red = img[:, :, 0]
green = img[:, :, 1]
blue = img[:, :, 2]

# Plot
fig, axs = plt.subplots(2,2, figsize=(12,12))


# Normal Image
cax_00 = axs[0,0].imshow(img)
axs[0,0].xaxis.set_major_formatter(plt.NullFormatter())  # kill xlabels
axs[0,0].yaxis.set_major_formatter(plt.NullFormatter())  # kill ylabels

# Red Channel
cax_01 = axs[0,1].imshow(red, cmap='Reds')
fig.colorbar(cax_01, ax=axs[0,1])
axs[0,1].xaxis.set_major_formatter(plt.NullFormatter())
axs[0,1].yaxis.set_major_formatter(plt.NullFormatter())


# Green Channel
cax_10 = axs[1,0].imshow(green, cmap='Greens')
fig.colorbar(cax_10, ax=axs[1,0])
axs[1,0].xaxis.set_major_formatter(plt.NullFormatter())
axs[1,0].yaxis.set_major_formatter(plt.NullFormatter())

# Blue Channel
cax_11 = axs[1,1].imshow(blue, cmap='Blues')
fig.colorbar(cax_11, ax=axs[1,1])
axs[1,1].xaxis.set_major_formatter(plt.NullFormatter())
axs[1,1].yaxis.set_major_formatter(plt.NullFormatter())

plt.show();

### Some basic information about the image

In [ ]:
# Size of one of the channels

red.shape

In [ ]:
# Example of the data

red[0]

### A (very) simple filter

Using linear algebra, we can transform the matrix

In [ ]:
red = red *1.2
blue = blue *1.4
green = green * 1.6

In [ ]:
# re-assign to the proper channel in the original image

img[:, :, 0] = red
img[:, :, 1] = green
img[:, :, 2] = blue

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)

## The MNIST Dataset

[This Wikipedia](https://en.wikipedia.org/wiki/MNIST_database) article covers the dataset in detail.

In [ ]:
# load dataset alhtough this returns train and test, we will only be using train
(trainX, trainy), (testX, testy) = mnist.load_data()

# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))

# plot first few images

plt.figure(figsize=(9,9))

for i in range(9):

    # define subplot
    plt.subplot(330 + 1 + i)

    # plot raw pixel data
    plt.imshow(trainX[i], cmap=plt.get_cmap('gray'))

# show the figure
plt.show()

In [ ]:
# The labels for the data set

trainy[:9]

In [ ]:
# Flatten the matrix into a feature vector for logistic regression

flat_trainX = np.reshape(trainX, (trainX.shape[0], (trainX.shape[1]**2)))

In [ ]:
flat_trainX.shape

In [ ]:
flat_trainX[0]

In [ ]:
# Normalize

flat_trainX = flat_trainX / 255.0

In [ ]:
flat_trainX[0]

## Logistic Regression?

For image recognition? Seriously?

In [ ]:
# Instantiate the logistic Regression

lr = LogisticRegression(max_iter=10000, n_jobs = -1)

In [ ]:
# Fit (train) the model to the training data
# Note: this take about 10 minutes on my machine (MacBookPro)

lr.fit(flat_trainX, trainy)

In [ ]:
# How accurate is it?

print(f'This model is {lr.score(flat_trainX, trainy):.2%} accurate.')

In [ ]:
# Create a dataframe predictions and actuals

df = pd.DataFrame(trainy, columns=['actual'])
df['preds'] = lr.predict(flat_trainX)

In [ ]:
# Create a dataframe with the probability predictions for each class, 
# and then multiply by 100 to create actual percentages

prob_col = ['prob'+str(i) for i in range(10)]
probs = (pd.DataFrame(lr.predict_proba(flat_trainX), columns=prob_col)) * 100

In [ ]:
# Concatenate the probabilities to the main dataframe

df = pd.concat([df, probs], axis = 1)

In [ ]:
# Create a sample of predictions that were wrong

wrong_sample = df[ df.actual != df.preds ].head(9)

In [ ]:
# plot first samples
i=0
plt.figure(figsize=(9,9))
for index in wrong_sample.index:
    
    # define subplot
    plt.subplot(330 + 1 + i)
    
    # plot raw pixel data
    plt.imshow(trainX[index], cmap=plt.get_cmap('gray'))
    
    # tight layout with spacing for title
    plt.tight_layout(pad=3.0)
    
    #get the prediction and actual value
    pred,actual = df.iloc[int(index)][['preds','actual']]
    
    #create the title
    plt.title(f'{index} Pred:{int(pred)} Actual:{int(actual)}')
    
    
    i += 1
    
    
# show the figure
plt.show()

In [ ]:
# Look at probability table

wrong_sample[prob_col]

### Sources

Some of the mnist code was derived from [this excellent](https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/) tutorial using the Tensorflow version of the dataset

The rbg separation was example was adapted from [this stackoverflow](https://stackoverflow.com/questions/39885178/how-can-i-see-the-rgb-channels-of-a-given-image-with-python) article